In [1]:
import open3d as o3d
import numpy as np
import math

C:\Users\grmc1\Anaconda3\envs\INNOVATE\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\grmc1\Anaconda3\envs\INNOVATE\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
C:\Users\grmc1\Anaconda3\envs\INNOVATE\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
from transformation import Transformation
from Signatures import Signature
from Utilities import *

In [3]:
p=o3d.io.read_point_cloud('D:\Documentos\INNOVATE\Base_de_datos\Recortes\LINEA__40_2_pulg_002.pts')
#o3d.visualization.draw_geometries([dp],point_show_normal=False)
dp=p.voxel_down_sample(voxel_size=0.007)
dp=p
dp.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

o3d.visualization.draw_geometries([dp],point_show_normal=False)


In [4]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])
cl, ind = dp.remove_statistical_outlier(nb_neighbors=5,std_ratio=1.5)
display_inlier_outlier(dp, ind)
dp=cl
dp=dp.voxel_down_sample(voxel_size=0.007)
dp.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
o3d.visualization.draw_geometries([dp])

Showing outliers (red) and inliers (gray): 


In [5]:
def upSampM1(pointcloud,KDT,NN,NNi=-1):
    points=np.asarray(pointcloud.points)
    P=random.randint(0,len(points)-1)
    point=pointcloud.points[P]
    [k, idx, q] = pt.search_knn_vector_3d(point,NN)
    zp=np.asarray(dp.points)[idx[:], :]
    npo=(zp[0,:]+zp[NNi,:])/2
    return npo

def upSampM2(pointcloud,KDT,NN,NNi=-1):
    points=np.asarray(pointcloud.points)
    P=random.randint(0,len(points)-1)
    point=pointcloud.points[P]
    [k, idx, q] = pt.search_knn_vector_3d(point,NN)
    zp=np.asarray(dp.points)[idx[:], :]
    p1=zp[0,:]
    p2=zp[NNi,:]
    d=np.linalg.norm(p1-p2)
    x=math.sqrt(d**2-(d/2)**2)
    pdp1=getPrincipalDir(dp,dp.points[P],pt,20)
    n1=pdp1[0][1]
    p12=p2-p1
    px=np.cross(p12,n1)
    px=px/(np.linalg.norm(px))
    p3=(p1+p2)/2+px*x
    return p3

def upsamplingM1(pointcloud,KDT,r,N):
    npts=np.empty((1,3))
    for i in range(N):
        p=upSampM1(pointcloud,KDT,r)
        npts=np.concatenate((npts,p.reshape(1,3)),axis=0)
    pointss=np.asarray(dp.points)
    pointss=np.concatenate((pointss,npts),axis=0)
    return pointss

def upsamplingM2(pointcloud,KDT,r,N):
    npts=np.empty((1,3))
    for i in range(N):
        p=upSampM2(pointcloud,KDT,r)
        npts=np.concatenate((npts,p.reshape(1,3)),axis=0)
    pointss=np.asarray(dp.points)
    pointss=np.concatenate((pointss,npts),axis=0)
    return pointss

def findIndex(pointarray,point_position):
    a=np.where(pointarray==point_position)
    return a[0][0]

In [5]:
P=3
NN=10
point=dp.points[P]
pt = o3d.geometry.KDTreeFlann(dp)
[k, idx, q] = pt.search_knn_vector_3d(point, NN)
zp=np.asarray(dp.points)[idx[:], :]
p1=zp[0,:]
p2=zp[-1,:]
d=np.linalg.norm(p1-p2)
x=math.sqrt(r**2-(d/2)**2)
pdp1=getPrincipalDir(dp,dp.points[P],pt,20)
n1=pdp1[0][1]
p12=p2-p1
px=np.cross(p12,n1)
px=px/(np.linalg.norm(px))
p3=(p1+p2)/2+px*x
[k, idx, q] = pt.search_knn_vector_3d(point, NN+3)
zp=np.asarray(dp.points)[idx[:], :]
zpp=zp-p3

zpn=np.linalg.norm(zpp,axis=1)
np.min(zpn)
a=np.where(zpn==np.min(zpn))
p3=zp[a[0][0]]
p23=p2-p3
p13=p1-p3
px=np.cross(p13,p23)
px=px/(np.linalg.norm(px))
p4=(p1+p2+p3)/3
p1i=findIndex(np.asarray(dp.points),p1)
p2i=findIndex(np.asarray(dp.points),p2)
p3i=findIndex(np.asarray(dp.points),p3)

pdp1=getPrincipalDir(dp,dp.points[p1i],pt,20)[0][1]
pdp2=getPrincipalDir(dp,dp.points[p2i],pt,20)[0][1]
pdp3=getPrincipalDir(dp,dp.points[p3i],pt,20)[0][1]
pdp4=pdp1+pdp2+pdp3
pdp4=pdp4/(np.linalg.norm(pdp4))

pointss=np.concatenate((np.asarray(dp.points),p4.reshape(1,3)),axis=0)
evalpcd=o3d.geometry.PointCloud()
evalpcd.points = o3d.utility.Vector3dVector(pointss)
evalpcd.estimate_normals()
estimated_normal=getPrincipalDir(evalpcd,evalpcd.points[-1],pt,20)[0][1]



NameError: name 'r' is not defined

In [6]:
estimated_normal=getPrincipalDir(evalpcd,evalpcd.points[-1],pt,20)[0][1]

NameError: name 'evalpcd' is not defined

In [7]:
print(pdp1)
print(pdp2)
print(pdp3)
print(pdp4)
print(estimated_normal)

NameError: name 'pdp1' is not defined

In [46]:
pt = o3d.geometry.KDTreeFlann(dp)
gpoints=upsamplingM2(dp,pt,4,500)
#Disminuir el numero NN
#Añadir limite de distancia mas cercano

In [47]:
print(gpoints.shape)
npcld=o3d.geometry.PointCloud()
npcld.points = o3d.utility.Vector3dVector(gpoints)
ao=np.ones((1004,3))*0.8
aa=np.ones((500,3))*0.2
colorss=np.concatenate((ao,aa),axis=0)
npcld.colors = o3d.utility.Vector3dVector(colorss)
o3d.visualization.draw_geometries([npcld],point_show_normal=False)


(1504, 3)
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


In [13]:
ogpts=gpoints[1024:]
npcld=o3d.geometry.PointCloud()
npcld.points = o3d.utility.Vector3dVector(ogpts)
o3d.visualization.draw_geometries([npcld],point_show_normal=False)

In [39]:
print(np.asarray(npcld.points).shape)
print(colorss.shape)


(1014, 3)
(1224, 3)
